# ATIVIDADE AVALIATIVA 01

In [ ]:
import pandas as pd
import re

dataset = pd.read_csv("https://raw.githubusercontent.com/viniciusrpb/cic0269_natural_language_processing/main/diario_oficial_pref_sp/dosp_29012016.csv")
dataset

In [ ]:
data_text = dataset[["Texto"]]
data_text.head()

In [ ]:
data_text.info()

## We want to extract three camps from Text:

- Título da Publicação
- Número do Processo
- Valor (Valor Global): valor real após "R$"

I looked at the entire file's content and I came to the following conclusion: 
<br>
<br>

- The title starts at least with "TITULO)" and ends at least with "(TEXTO)" or \n, being case insensitive

<br>
<br>

I say that because there are some titles starting with "TITULO)" or "(TITULO) and there are some "TITULO" words in TEXTO

<br>

And there are not "(TEXTO)" words in TEXTO 


In [ ]:
data_text.Texto[0]

## Extracting the Title Camp

In [ ]:
pattern1 = r"(^(?:\(?\(?T[ÍÌI]T[ÚÙU]LO\)).+(?:\(?\(TEXTO\)\)?))"  ## Pegar o TITULO e o restante do conteúdo até o TEXTO
pattern2 = r"(.+)" ## Pegar o TITULO até o \n mais próximo, se tiver
pattern3 = r"\(?\(?T[ÍÌI]T[ÚÙU]LO\)\)?|\(?\(TEXTO\)\)?|\(?\(CL\)\)?|\(?\(NG\)\)?"  ## Substituindo as tags (TITULO) E (TEXTO) E (NG) e (CL)

# pattern1 = r"(^(?:\(?\(?T[ÍÌI]T[ÚÙU]LO\)).+(?:\(?\(TEXTO\)\)?))"  ## Pegar o TITULO e o restante do conteúdo até o TEXTO, Prevenir esse caso aki ((TITULO))((TEXTO))\n\n \n((TITULO))
# pattern2 = r"[(?:(?:\(?\(TEXTO\)\))|(?:\(?\(?T[ÍÌI]T[ÚÙU]LO\)))]*(.+)[(?:(?:\(?\(TEXTO\)\))|(?:\(?\(?T[ÍÌI]T[ÚÙU]LO\)))]*"
# pattern3 = r"(.+)(?:\(?\(TEXTO\)\)?)" ## Pegar o TITULO até o \n mais próximo, se tiver
# pattern4 = r"\(?\(?T[ÍÌI]T[ÚÙU]LO\)\)?|\(?\(TEXTO\)\)?|\(?\(CL\)\)?|\(?\(NG\)\)?"  ## Substituindo as tags (TITULO) E (TEXTO) E (NG) e (CL)

## TESTE
# result = re.search(pattern1, "((TITULO))((TEXTO))\n\n \n((TITULO))GABINETE DO SUBPREFEITO\n\n \n\n((TEXTO))DESPACHO\n\n PA:2016-0.018.185-2\n\nINTERESSADO: SUBPREFEITURA PERUS\nASSUNTO: ((NG))AUTORIZAÇÃO ((CL))PARA EMPENHAMENTO DE AUXÍLIO REFEIÇÃO\n\nI - DESPACHO:\n\n \n 1- No uso da competência a mim delegada pela Lei Municipal 13.399/2002 art. 9º, XX e XXI, \n\ne com fundamento na Lei 13.598 de 05 de junho de 2003,((NG)) AUTORIZO((CL)) a despesa e emissão das respectivas \n\nNotas de Empenho, no valor estimativo de R$ 750.900,21 (setecentos e cinquenta mil e novecentos reais e vinte e um \n\ncentavos), bem como as complementações que se fizerem necessárias durante o exercício de 2016, para cobertura das \n\ndespesas com AUXILIO REFEIÇÃO, a ser utilizado pelos servidores desta SUBPREFEITURA PERUS CNPJ 00.050.231/0000-00, \n\n onerando a dotação 41.10.15.122.3024.2100.3.3.90.46.00 no presente exercício, em conformidade com a liberação das \n\ncotas orçamentárias/financeiras que viere...	",flags=re.I | re.DOTALL)
# print(result.group(), end="__________")
# result = re.search(pattern2, result.group(), flags=re.I | re.DOTALL)
# print(result.group(), end="__________")
# result = re.search(pattern3, result.group(), flags=re.I)
# print(result.group(), end="__________")
# result = re.sub(pattern4, "", result.group(), flags=re.I)

# print(result)

In [ ]:
data_text["Title"] = data_text.Texto.str.extract(pattern1, flags=re.I | re.DOTALL) ## Pegando o Título
data_text["Title"] = data_text.Title.str.extract(pattern2, flags=re.I) ## Pegando até o \n
data_text["Title"] = data_text.Title.str.replace(pattern3, "", regex=True, flags=re.I)
data_text["Title"] = data_text.Title.str.replace(pattern3, "", regex=True, flags=re.I).str.strip()
data_text.Title[data_text.Title.str.len() == 0] = "NaN"  ## Replacing "" Titles, aparece quando tem ((TITULO))((TEXTO))

data_text

## Extracting the Process's Number

Pelo o que eu vi, muitos números de processo vêm acompanhados da palavra processo ou semelhantes

In [ ]:
## Testando
pattern1 = r"[^(?:processo)]*[(?:processo)|(?:processo)|(?:pa)|(?:proc)]+[^0-9|(?:DE)]{0,25}([0-9|\.|\-|/]{7,25})"  ## Para pegar os que tem a palavra processo do lado e semelhantes
pattern2 = r"[^0-9]"

# ## Teste nos pattern 1 e pattern 2
# result = re.search(pattern1, "Dwhaudiwad Processo Sei 31928301-12391/23\n\n", flags=re.I)
# result = re.sub(pattern2, "", result.group())

In [ ]:
data_text["Process"] = data_text.Texto.str.extract(pattern1, flags=re.I)
data_text["Process"] = data_text.Process.str.replace(pattern2, "").str.strip()
data_text.Process[data_text.Process.str.len() == 0] = "NaN"
data_text[["Texto", "Process"]]

## Extracting Valor (Valor Global): valor real após "R$"

In [ ]:
## Testando
# pattern1 = r".*Valor\s+Global\s+R$\s+([0-9|\.|\-|/|,]+)[^0-9|\.|\-|/|,]"  ## Para pegar os que tem a palavra processo do lado e semelhantes
pattern1 = r".*(?:VALOR)+\s*(?:GLOBAL)+\s*[^$]+\$\s*([0-9|\.|,]+)"  ## Para pegar os que tem a palavra processo do lado e semelhantes
pattern2 = r"([^0-9|\.|,]+)"

# ## Teste nos pattern 1 e pattern 2
# result = re.search(pattern1, "((NG))VALOR GLOBAL: ((CL))R$ 60.000,00 (sessenta mil reais), a serem pagos em até 30 (trinta) dias, após a confirmação da execução dos serviços pela unidade requisitante.", flags=re.I | re.DOTALL)
# print(result.group())
# result = re.sub(pattern2, "", result.group())
# print(result)

In [ ]:
data_text["Global_Value"] = data_text.Texto.str.extract(pattern1, flags=re.I | re.DOTALL).str.strip()
# data_text["Global_Value"] = data_text.Texto.str.replace(pattern2, "", flags=re.I | re.DOTALL)
data_text.Global_Value[data_text.Global_Value.str.len() == 0] = "NaN"
# data_text[["Texto", "Global_Value"]]